In [1]:
ENV["PYTHON"] = "" 
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
using PyPlot
ENV["PYTHON"] = "" 


    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/b19db3927f0db4151cb86d073689f2428e524576/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`


LoadError: InitError: KeyError: key "register_cmap" not found
during initialization of module PyPlot

In [2]:
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
using PyPlot
using DelimitedFiles
using DCAUtils
using KitMSA
using FastaIO
using Statistics
using StatsBase
using LinearAlgebra
using CSV, DataFrames

include("../../Gen.jl/src/read_write.jl")
include("../../Gen.jl/src/energy.jl")
include("../../Gen.jl/src/msa_tools.jl")
include("../../Gen.jl/src/extra_tools.jl")
;


function extract_1wt(input_df, prot)
    wt = []
    df = input_df[input_df[:, "PROT"] .== prot,:]
    site_vec = df.SITE
    l = length(site_vec)
    for (i, s) in enumerate(site_vec[1:(l - 1)])
        if site_vec[i+1] != site_vec[i] || i == (l - 1)
            append!(wt, df.WTAA[i])
        end
    end
    res = []
    for i in 1:length(wt)
        push!(res, Int8.(letter2num(wt[i])))
    end
    return res
end

df = CSV.read("../../Gen.jl/data/alignments/refs/Thornton.csv", DataFrame)
L = -2.1482 # lower bound of measurement
R =  1.6081 # dynamic range of measurement
U = L + R # upper bound of measurement
WT_ACTIVITY = -0.79 # standard WT activity
LB = L - WT_ACTIVITY + 0.05
UB = U - WT_ACTIVITY - 0.05

df[:,5] = df[:,5] .- WT_ACTIVITY
sel_wt_MAP_1 =    [1,    10,    19, 6, 7, 8, 12] # maximum a posteriori reconstructed wt
sel_wt_MAP_2 =    [1, 10,    4, 14]
;

l2n = [num2letter(i) for i in 1:20]
wts1 = Int8.(hcat([extract_1wt(df, prot) for prot in sel_wt_MAP_1]...))

N_wt = length(sel_wt_MAP_1)

for i in 1:N_wt
    exp_dms = []
    exp_prot = []
    wt_ind = sel_wt_MAP_1[i]
    dms = df[  df[:, "PROT"] .== wt_ind, :]
    N_mut = length(dms[:,1])
    for n in 1:N_mut
        aa = Int8(findfirst(x -> x == dms.MTAA[n], l2n))
        new_wt = deepcopy(wts1[:,i])
        new_wt[dms.SITE[n]] = aa
        push!(exp_prot, new_wt)
        push!(exp_dms, dms.FIT[n])
    end
    matrix2fasta("../../dms_thornton_proteins_wt$(sel_wt_MAP_1[i]).fasta", Int8.(hcat(exp_prot...)'))
    writedlm("../../dms_thornton_fitness_wt$(sel_wt_MAP_1[i]).txt", exp_dms)
end



    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/b19db3927f0db4151cb86d073689f2428e524576/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`
